# **PROBLEMA DE NEGOCIO**


---




### Una tienda online de moda, con presencia en todo Brasil, necesita impulsar su rendimiento utilizando sus datos de manera estratégica. Como científico de datos, has sido convocado para analizar estos datos y ofrecer insights que guíen sus decisiones y respondan a algunas preguntas claves

# **1. Configuración del Ambiente**


---




In [207]:
# !pip install geobr
import geobr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.patches import FancyBboxPatch
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
import matplotlib.dates as mdates
from matplotlib.ticker import FuncFormatter
import matplotlib.ticker as ticker
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, Float, Date, ForeignKey, text
from sqlalchemy.orm import sessionmaker

# **2. Obtención, Tratamiento y Análisis Exploratório (EDA)**


---




## **2.1 Cargando las bases de datos**

In [208]:
# Carga de datos locales
local = True

In [209]:
if local:
    df_itens_pedidos = pd.read_csv('itens_pedidos.csv')
    df_pedidos = pd.read_csv('pedidos.csv')
    df_productos = pd.read_csv('productos.csv')
    df_vendedores = pd.read_csv('vendedores.csv')

In [210]:
if not local:
    df_itens_pedidos = pd.read_csv('https://raw.githubusercontent.com/ElProfeAlejo/Bootcamp_Databases/main/itens_pedidos.csv')
    df_itens_pedidos.to_csv('itens_pedidos.csv', index=False)

In [211]:
df_itens_pedidos

,id_recibo,producto_id,pedido_id,cantidad,valor_unitario,valor_total,ciudad,costo_envio
0,1,41518,341,3,260,780,BR-BA,156.0
1,2,4307,1174,5,6175,30875,BR-RJ,6175.0
2,3,22407,1399,3,200,600,BR-PB,120.0
3,4,3820,1652,6,139,834,BR-DF,166.8
4,5,29012,2470,3,525,1575,BR-BA,315.0
...,...,...,...,...,...,...,...,...
24522,25051,64127,63716,4,590,2360,BR-PE,472.0
24523,25052,37384,63999,4,700,2800,BR-RO,560.0
24524,25053,100932,64594,5,3900,19500,BR-RO,3900.0
24525,25054,3820,64811,3,139,417,BR-MA,83.4


In [212]:
if not local:    
    df_pedidos = pd.read_csv('https://raw.githubusercontent.com/ElProfeAlejo/Bootcamp_Databases/main/pedidos.csv')
    df_pedidos.to_csv('pedidos.csv', index=False)

In [213]:
df_pedidos

,pedido_id,producto_id,vendedor_id,fecha_compra,total
0,341,41518,6,2019-01-05,780
1,1174,4307,3,2019-01-05,30875
2,1399,22407,1,2019-01-05,600
3,1652,3820,4,2019-01-05,834
4,2470,29012,2,2019-01-05,1575
...,...,...,...,...,...
24522,63716,64127,1,2021-03-09,2360
24523,63999,37384,2,2021-03-09,2800
24524,64594,100932,1,2021-03-10,19500
24525,64811,3820,3,2021-03-10,417


In [214]:
if not local:
    df_productos = pd.read_csv('https://raw.githubusercontent.com/ElProfeAlejo/Bootcamp_Databases/main/productos.csv')
    df_productos.to_csv('productos.csv', index=False)

In [215]:
df_productos

,producto_id,producto,precio,marca,sku,condicion
0,21244,Oculos Lente Azulada,1120,D&g Dolce & Gabbana,209297.0,Usado
1,9981,Bolsa Coral Saco,4000,Givenchy,278612.0,Usado
2,84176,Camisa Xadrez Verde,310,Joe Fresh,322482.0,Usado
3,47475,Calca Alfaiataria Preta,490,Mixed,263658.0,Usado
4,74864,Vestido Jeans Babados,130,Zara,219248.0,Usado
...,...,...,...,...,...,...
202,22568,Casaqueto Estampa Geometrica,245,Zara,244931.0,Usado
203,40508,Regata Bicolor Alcinha,245,Express,247166.0,Usado
204,41518,Calca Jeans Basica,260,Banana Republic,255410.0,Nuevo con etiqueta
205,41598,NaN,260,Banana Republic,NaN,Nuevo con etiqueta


In [216]:
if not local:
    df_vendedores = pd.read_csv('https://raw.githubusercontent.com/ElProfeAlejo/Bootcamp_Databases/main/vendedores.csv')
    df_vendedores.to_csv('vendedores.csv', index=False)

In [217]:
df_vendedores

,vendedor_id,nombre_vendedor
0,1,Ana Duarte
1,2,Daniel Siqueira
2,3,Nadia Oliveira
3,4,Millena Pereira
4,5,Paulo Calanca
5,6,Unknown


## **2.2 Analisis, Observaciones y Conclusiones**

In [218]:
print(df_itens_pedidos.info())
print('-------------')
print(df_pedidos.info())
print('-------------')
print(df_productos.info())
print('-------------')
print(df_vendedores.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24527 entries, 0 to 24526
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id_recibo       24527 non-null  int64  
 1   producto_id     24527 non-null  int64  
 2   pedido_id       24527 non-null  int64  
 3   cantidad        24527 non-null  int64  
 4   valor_unitario  24527 non-null  int64  
 5   valor_total     24527 non-null  int64  
 6   ciudad          24527 non-null  object 
 7   costo_envio     24527 non-null  float64
dtypes: float64(1), int64(6), object(1)
memory usage: 1.5+ MB
None
-------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24527 entries, 0 to 24526
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   pedido_id     24527 non-null  int64 
 1   producto_id   24527 non-null  int64 
 2   vendedor_id   24527 non-null  int64 
 3   fecha_compra  24527 non-null  object
 

---

Vendedores

In [219]:
df_pedidos[df_pedidos['vendedor_id'] == 6]

,pedido_id,producto_id,vendedor_id,fecha_compra,total
0,341,41518,6,2019-01-05,780


In [220]:
df_itens_pedidos[df_itens_pedidos['pedido_id'] == 341]

,id_recibo,producto_id,pedido_id,cantidad,valor_unitario,valor_total,ciudad,costo_envio
0,1,41518,341,3,260,780,BR-BA,156.0


In [221]:
df_pedidos[df_pedidos['pedido_id'].isin(df_itens_pedidos[df_itens_pedidos['ciudad']=='BR-BA']['pedido_id'])]

,pedido_id,producto_id,vendedor_id,fecha_compra,total
0,341,41518,6,2019-01-05,780
4,2470,29012,2,2019-01-05,1575
16,6355,3820,5,2019-01-11,556
21,10809,12609,1,2019-01-14,1600
34,16923,4553,5,2019-01-17,220
...,...,...,...,...,...
24461,19227,83866,4,2021-02-20,990
24476,28011,29893,4,2021-02-20,840
24487,35555,9809,3,2021-02-20,1560
24494,41529,12812,2,2021-02-21,1280


In [222]:
df_pedidos[df_pedidos['fecha_compra']=='2019-01-05']

,pedido_id,producto_id,vendedor_id,fecha_compra,total
0,341,41518,6,2019-01-05,780
1,1174,4307,3,2019-01-05,30875
2,1399,22407,1,2019-01-05,600
3,1652,3820,4,2019-01-05,834
4,2470,29012,2,2019-01-05,1575
5,2980,72621,1,2019-01-05,537
6,3443,86243,3,2019-01-05,1400
7,3557,29012,4,2019-01-05,2625
8,3585,61071,1,2019-01-05,1350


itens_pedidos

In [223]:
df_productos[['producto', 'marca', 'condicion']].value_counts()

producto                   marca        condicion         
Saia Pregas Preta          Zara         Usado                 2
Bata Pink Decote           Loft 747     Nuevo con etiqueta    1
Sandalia Prata Pedraria    Thay Ribas   Usado                 1
Saia Pregas Azul           Carina Duek  Nuevo con etiqueta    1
Saia Preta Babado          Mixed        Usado                 1
                                                             ..
Camisa Acetinada Mostarda  Thelure      Usado                 1
Camisa Amarela Laco        Carol Bassi  Usado                 1
Camisa Fio Animalprint     Bobô         Usado                 1
Camisa Jeans Azul          Zara         Usado                 1
Vestido Ziper Fendi        Thelure      Usado                 1
Name: count, Length: 204, dtype: int64

In [224]:
df_productos[df_productos['precio']==210]

,producto_id,producto,precio,marca,sku,condicion
22,23575,Camiseta Animal Print,210,284,283163.0,Usado
72,70673,Shorts Listras Bordados,210,American Eagle,126190.0,Usado
86,45878,Vestido Preto Bolso,210,Zara,114614.0,Usado
149,50330,Camisa Jeans Azul,210,Zara,250096.0,Usado
152,5567,Vestido Medio Florl,210,Amaro,525428.0,Usado
164,65473,Oculos Acetato Branco,210,Mango,944049.0,Usado
183,32126,Saia Midi Cinto,210,Limone By Jade Seba,302628.0,Nuevo sin etiqueta
206,32926,NaN,210,Limone By Jade Seba,NaN,Nuevo sin etiqueta


In [225]:
df_productos.producto_id.nunique()

207

In [226]:
df_productos[df_productos['producto'].isna()]

,producto_id,producto,precio,marca,sku,condicion
205,41598,NaN,260,Banana Republic,NaN,Nuevo con etiqueta
206,32926,NaN,210,Limone By Jade Seba,NaN,Nuevo sin etiqueta


In [227]:
df_itens_pedidos[df_itens_pedidos['producto_id']==41598]

,id_recibo,producto_id,pedido_id,cantidad,valor_unitario,valor_total,ciudad,costo_envio


In [228]:
df_itens_pedidos[df_itens_pedidos['producto_id']==32926]

,id_recibo,producto_id,pedido_id,cantidad,valor_unitario,valor_total,ciudad,costo_envio


pedidos

In [229]:
df_pedidos

,pedido_id,producto_id,vendedor_id,fecha_compra,total
0,341,41518,6,2019-01-05,780
1,1174,4307,3,2019-01-05,30875
2,1399,22407,1,2019-01-05,600
3,1652,3820,4,2019-01-05,834
4,2470,29012,2,2019-01-05,1575
...,...,...,...,...,...
24522,63716,64127,1,2021-03-09,2360
24523,63999,37384,2,2021-03-09,2800
24524,64594,100932,1,2021-03-10,19500
24525,64811,3820,3,2021-03-10,417


* ``df_vendedores``: vendedor_id 6 sin Nombre
    * Una venta el 2019-01-05 por valor 780
    * No pudimos inferir sobre quien es el vendedor en cuestion
    * **Solucion**: no tenerlo en cuenta para vendedores individuales
* ``df_itens_pedidos``: ademas ver productos
    * "ciudad" esta en un formato que no es estandard para geobr
    * **Solucion**: quitaremos BR_
* ``df_pedidos``: "fecha_compra" a tipo datetime
* ``df_productos``: no hay pedidos con los productos que no sabemos su nombre:
    * De todas formas podemos inferir que el primer producto (41598) es "Calca Jeans Basica" y el segundo producto (32926) es "Saia Midi Cinto" de acuerdo a su precio y marca
    * A su vez sus "SKU" (Stock Keeping Unit) son 255410 y 302628 respectivamente
    * **Solucion**: eliminarlos por estar mal registrados (duplicados)

---

## **2.3 Tratamiento de Datos**

In [230]:
def preprocesamiento():
  df_productos.dropna(subset=['producto'], inplace=True)
  df_itens_pedidos['ciudad'] = df_itens_pedidos['ciudad'].str.replace('BR-', '')
  df_pedidos['fecha_compra'] = pd.to_datetime(df_pedidos['fecha_compra'])

In [231]:
preprocesamiento()
print(df_itens_pedidos.info())
print('-------------')
print(df_pedidos.info())
print('-------------')
print(df_productos.info())
print('-------------')
print(df_vendedores.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24527 entries, 0 to 24526
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id_recibo       24527 non-null  int64  
 1   producto_id     24527 non-null  int64  
 2   pedido_id       24527 non-null  int64  
 3   cantidad        24527 non-null  int64  
 4   valor_unitario  24527 non-null  int64  
 5   valor_total     24527 non-null  int64  
 6   ciudad          24527 non-null  object 
 7   costo_envio     24527 non-null  float64
dtypes: float64(1), int64(6), object(1)
memory usage: 1.5+ MB
None
-------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24527 entries, 0 to 24526
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   pedido_id     24527 non-null  int64         
 1   producto_id   24527 non-null  int64         
 2   vendedor_id   24527 non-null  int64         
 3 

## **2.4 Creando el Banco de Datos y las Herramientas para Utilizarlo**

In [232]:
# Crearemos una base de datos sql con sqlalchemy
def crear_conexion():
  # Crear conexión a la base de datos
  engine = create_engine('sqlite:///tienda.db', echo=True)
  return engine

engine = crear_conexion()

In [233]:
# Creamos Tbalas
def crear_tablas(engine):
  # Crear las tablas
  metadata = MetaData()
  Table('productos', metadata,
        Column('producto_id', Integer, primary_key=True),
        Column('producto', String),
        Column('marca', String),
        Column('precio', Float),
        Column('condicion', String)
        )
  Table('vendedores', metadata,
        Column('vendedor_id', Integer, primary_key=True),
        Column('nombre', String),
        Column('ciudad', String)
        )
  Table('pedidos', metadata,
        Column('pedido_id', Integer, primary_key=True),
        Column('fecha_compra', Date),
        Column('vendedor_id', Integer, ForeignKey('vendedores.vendedor_id'))
        )
  Table('itens_pedidos', metadata,
        Column('item_id', Integer, primary_key=True),
        Column('pedido_id', Integer, ForeignKey('pedidos.pedido_id')),
        Column('producto_id', Integer, ForeignKey('productos.producto_id')),
        Column('cantidad', Integer),
        Column('precio', Float),
        Column('ciudad', String)
        )
  metadata.create_all(engine)

crear_tablas(engine)

2024-04-09 22:45:25,721 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-09 22:45:25,721 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("productos")
2024-04-09 22:45:25,722 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-09 22:45:25,724 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("vendedores")
2024-04-09 22:45:25,725 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-09 22:45:25,726 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("pedidos")
2024-04-09 22:45:25,727 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-09 22:45:25,728 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("itens_pedidos")
2024-04-09 22:45:25,729 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-09 22:45:25,730 INFO sqlalchemy.engine.Engine COMMIT


In [234]:
def cargar_datos(engine):
  # Cargar los datos
  df_productos.to_sql('productos', con=engine, if_exists='replace', index=False)
  df_vendedores.to_sql('vendedores', con=engine, if_exists='replace', index=False)
  df_pedidos.to_sql('pedidos', con=engine, if_exists='replace', index=False)
  df_itens_pedidos.to_sql('itens_pedidos', con=engine, if_exists='replace', index=False)

cargar_datos(engine)

2024-04-09 22:45:25,742 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-09 22:45:25,745 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("productos")
2024-04-09 22:45:25,747 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-09 22:45:25,749 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("productos")
2024-04-09 22:45:25,750 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-09 22:45:25,752 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2024-04-09 22:45:25,753 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-09 22:45:25,755 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='view' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2024-04-09 22:45:25,756 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-09 22:45:25,757 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("productos")
2024-04-09 22:45:25,758 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-0

In [235]:
# Creamos la funcion de consulta
def consultar(query, engine):
    # Crear una sesión
    Session = sessionmaker(bind=engine)
    session = Session()

    try:
        # Ejecutar la consulta
        result = session.execute(text(query))
        
        # Devolver los resultados
        return result.fetchall()
    except Exception as e:
        print(f"Ocurrió un error al ejecutar la consulta: {e}")
    finally:
        # Cerrar la sesión
        session.close()